In [1]:
# 종합: 데이터 불러오기, 저장하기, 분석하기

In [1]:

import requests
import pandas as pd
import numpy as np
from io import BytesIO
import re
import os
from zipfile import ZipFile
import xmltodict
import json

api_key = '7d523a1a3ce98201eb9eeb5c04c206363c176810'
url_corp = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={}'.format(api_key)
resp_corp = requests.get(url_corp)

zf_corp = ZipFile(BytesIO(resp_corp.content))
zf_corp_name = zf_corp.namelist()
zf_corp_name_pop = zf_corp_name.pop()

data_xml = zf_corp.read(zf_corp_name_pop).decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')



In [5]:
def dload_dart_10q(corp_name):

    where_name = (item for item in data if item['corp_name'] == corp_name)
    # corp_name 이 corp_name인 dict를 list에서 가져오기
    # 만약 값이 없을 경우 false가 dict에 저장됨
    dict_name = next(where_name, False)
    corp_code = dict_name['corp_code']

    print(corp_code)
    
    url_info = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de=19990101&last_reprt_at=N&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003'.format(api_key, corp_code)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
    headers = {'user-agent' : user_agent}

    resp = requests.get(url_info)
    resp_decode = resp.content.decode('utf-8')

    period = re.findall(r'<report_nm>(.*?)</report_nm>', resp_decode)
    rcp = re.findall(r'<rcept_no>(.*?)</rcept_no>', resp_decode)
    # <report_nm> 이런건 미리 해보고 알 수 있는 것들
    # company는 아직... 좀 더 해보고...

    dcm = []

    for i in range(len(rcp)):
        url_rcp = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rcp[i]
        resp_rcp = requests.get(url_rcp, headers = headers)
        resp_rcp_decode = resp_rcp.content.decode('utf-8')
        dcm_i = re.findall(rcp[i]+"', '(.*?)',", resp_rcp_decode)[0]
        dcm = dcm + [dcm_i]
        
    dict2_info = dict(zip(zip(period, rcp),dcm))
    
    print(period[0])
    print(rcp[0])
    print(dcm[0])
    
    os.makedirs('./'+corp_name)
    
    for i in range(min(len(period), len(rcp), len(dcm))):
        url_rcpdcm = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no='+rcp[i]+'&dcm_no='+dcm[i]+'&lang=ko'
               
        resp_rcpdcm = requests.get(url_rcpdcm, headers = headers)
        output = BytesIO(resp_rcpdcm.content)
        sheet_name_s = list(pd.read_excel(output, sheet_name = None).keys())

        for sheet in sheet_name_s:
            output = BytesIO(resp_rcpdcm.content)
            table = pd.read_excel(output, sheet_name = sheet, skiprows = 5)
            table.to_csv('./'+corp_name+'/'+period[i]+' '+corp_name+' '+sheet+".csv", encoding = 'euc-kr')
    print(sheet_name_s)

In [24]:
dload_dart_10q('SK텔레콤')

00159023
분기보고서 (2020.09)
20201116000869
7548156
['기본정보', '연결 재무상태표', '연결 손익계산서', '연결 포괄손익계산서', '연결 자본변동표', '연결 현금흐름표', '재무상태표', '손익계산서', '포괄손익계산서', '자본변동표', '현금흐름표']


In [29]:
def pre_chart_data(corp_name, fs_type, fs_item):
    pl_s = []
    period_s = []
    rev_s = []

    for file in os.listdir('./'+corp_name):
        if fs_type in file:
            pl_s.append(file) 

            period = re.findall(r'{}(.*?){}'.format('보고서 \(', '\)'), file)
            period_s.append(period[0])

    for file in pl_s:

        df = pd.read_csv('./'+corp_name+'/{}'.format(file), encoding = 'euc-kr')
        df1 = df.apply(lambda x : x.str.contains(fs_item), axis = 1)
        
        if df1.sum().sum() == 1 :
            
            col = df1.sum(axis = 0).values.argmax()
            row = df1.sum(axis = 1).values.argmax()
            rev = df.iloc[row, col+1]
            rev_s.append(rev)
            
        else:
            print(df1.sum().sum())
        
    period_ss = list(map(float, period_s))
    rev_ss = list(map(int, rev_s))
    rev_table = dict(zip(period_ss, rev_ss))
    
    rev_table_sort = sorted(rev_table.items())
    
    period_sss = []
    rev_sss = []

    for i in range(len(rev_table_sort)):
        period_sss_i = rev_table_sort[i][0]
        rev_sss_i = rev_table_sort[i][1]

        period_sss.append(str(period_sss_i))
        rev_sss.append(rev_sss_i)

    rev_table_sorted = dict(zip(period_sss, rev_sss))
    return rev_table_sorted

In [32]:
pre_chart_data('LG전자', '연결 손익계산서', '매출액')

{'2018.06': 15019311,
 '2018.09': 15427050,
 '2018.12': 61341664,
 '2019.03': 14915064,
 '2019.06': 15629264,
 '2019.09': 15700645,
 '2019.12': 62306175,
 '2020.03': 14727803,
 '2020.06': 12833827,
 '2020.09': 16919559}

In [ ]:
### 문제점! : 삼성전자는 매출액이 수익(매출액)으로 표시되어 있음! 빈칸이 또 방해하기도 함!

In [15]:


pl_s = []
period_s = []
rev_s = []

for file in os.listdir('./현대자동차'):
    if '연결 손익계산서' in file:
        pl_s.append(file) 

        period = re.findall(r'{}(.*?){}'.format('보고서 \(', '\)'), file)
        period_s.append(period[0])

for file in pl_s:

    df = pd.read_csv('./현대자동차/{}'.format(file), encoding = 'euc-kr')

    df_boolean = df == '매출액'
    col = df_boolean.sum(axis = 0).values.argmax()
    row = df_boolean.sum(axis = 1).values.argmax()
    rev = df.iloc[row, col+1]
    rev_s.append(rev)

period_ss = list(map(float, period_s))
rev_ss = list(map(int, rev_s))
rev_table = dict(zip(period_ss, rev_ss))

rev_table_sort = sorted(rev_table.items())

period_sss = []
rev_sss = []

for i in range(len(rev_table_sort)):
    period_sss_i = rev_table_sort[i][0]
    rev_sss_i = rev_table_sort[i][1]

    period_sss.append(str(period_sss_i))
    rev_sss.append(rev_sss_i)

rev_table_sorted = dict(zip(period_sss, rev_sss))
rev_table_sorted

{'2018.06': 24711802,
 '2018.09': 24433735,
 '2018.12': 96812609,
 '2019.03': 23987079,
 '2019.06': 26966353,
 '2019.09': 26968853,
 '2019.12': 105746422,
 '2020.03': 25319448,
 '2020.06': 21858991,
 '2020.09': 27575812}

In [17]:
df.head()

,Unnamed: 0,(단위 : 백만원),Unnamed: 1,Unnamed: 2,Unnamed: 3
0,0,,제 52 기,제 51 기,제 50 기
1,1,매출액,105746422,96812609,96376079
2,2,매출원가,88091409,81670479,78798172
3,3,매출총이익,17655013,15142130,17577907
4,4,판매비와관리비,14049508,12719965,13003240


In [23]:
df.apply(lambda x : x.str.strip(), axis = 1)

,Unnamed: 0,(단위 : 백만원),Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,,제 52 기,제 51 기,제 50 기
1,NaN,매출액,105746422,96812609,96376079
2,NaN,매출원가,88091409,81670479,78798172
3,NaN,매출총이익,17655013,15142130,17577907
4,NaN,판매비와관리비,14049508,12719965,13003240
5,NaN,영업이익,3605505,2422165,4574667
6,NaN,공동기업및관계기업투자손익,542826,404541,225053
7,NaN,금융수익,827120,823499,972943
8,NaN,금융비용,475218,600867,1120386
9,NaN,기타수익,1120958,967281,1153744
